In [ ]:




from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install stanza
!pip install textacy
import os
import spacy
import textacy
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk import RegexpParser
import stanza
import re
corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)
os.environ["CORENLP_HOME"] = corenlp_dir

     |████████████████████████████████| 574 kB 5.2 MB/s 
     |████████████████████████████████| 175 kB 18.0 MB/s 
     |████████████████████████████████| 4.0 MB 30.9 MB/s 
     |████████████████████████████████| 596 kB 35.5 MB/s 
     |████████████████████████████████| 77 kB 6.2 MB/s 
     |████████████████████████████████| 880 kB 58.1 MB/s 
     |████████████████████████████████| 6.6 MB 46.2 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=1949731b5bdf81c18bdac403e6a1d3a457b56aa475aa5d11716991d638c7e18f
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=ea910668b1459fe3a69d3f1506eaf3b2a2be71ada0ed7ceca0144c8acd1de78d
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built emoji sacremoses
  Attempting uninstall: pyyaml
    Fou

2022-05-10 05:19:15 INFO: Installing CoreNLP package into ./corenlp


2022-05-10 05:19:24 WARNING: For customized installation location, please set the `CORENLP_HOME` environment variable to the location of the installation. In Unix, this is done with `export CORENLP_HOME=./corenlp`.


In [ ]:
import json

In [ ]:
import

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

     |████████████████████████████████| 12.8 MB 5.1 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
def generate_phrases(file):
    nlp = spacy.load("en_core_web_sm")#part of import statements
    with open(file,'r',encoding="utf-8-sig") as f:#open and read the txt file
        data = f.read().replace('\n',' ')
    
    spacy_sent = list(nlp(data).sents)#gets all the sentences from the doc in a type spacy.
    sentences = []
    for s in spacy_sent:
      x = s.orth_.strip()
      if len(x)!=0:
        sentences.append(x)#orth_ gives the sentences in str form.

    print("Sentences : ",sentences)

    result = []
    
    for s in sentences:
      np = nlp(s).noun_chunks
      np = [t.text for t in np]
      result.append((s,np))          
    return (" ".join(sentences),result)

In [ ]:
def generate_phrases1(file):   # donot execute this function
    nlp = spacy.load("en_core_web_sm")#part of import statements
    with open(file,'r',encoding="utf-8-sig") as f:#open and read the txt file
        data = f.read().replace('\n',' ')
    
    spacy_sent = list(nlp(data).sents)#gets all the sentences from the doc in a type spacy.
    sentences = []
    for s in spacy_sent:
      x = s.orth_.strip()
      if len(x)!=0:
        sentences.append(x)#orth_ gives the sentences in str form.

    print("Sentences : ",sentences)

    result = []
    
    for s in sentences:
      np = nlp(s).noun_chunks
      np = [t.text for t in np]
      result.append((s,np))          
    return (" ".join(sentences),result)

In [ ]:
def getAnnotatedDocument(filePath):
  text,sentence_np = generate_phrases(filePath)
  ##text,sentence_pp = generate_phrases1(filepath)
  from stanza.server import CoreNLPClient
  print("Starting server with the Python...")
  document = []
  with CoreNLPClient(annotators=['tokenize','ssplit', 'pos', 'lemma', 'ner', 'coref'], 
                   memory='5G', endpoint='http://localhost:9001', be_quiet=True,timeout= 80000) as client:
    document = client.annotate(text)
    document.entityMentionToCorefMentionMappings
  print("Closing server...")
  #### try to separarate sentences here #############
  return (document,text,sentence_np)

In [ ]:
def getLengthSentencesAndNouns(document,verbose=False):
  SENTENCES_LENGTH = len(document.sentence)
  #totalphrase=[] this works but gives same output So no use
  #print(sent1)
  if verbose:
    print("here we are printing value of sents")
    # print(sent1)   ###this works 
    #sent11=[]
    #for sents11 in enumerate(document.sentence) 
    #sent11.append
  #print(sent11)
  
  if verbose:
    print("{:12s}\t{:12s}\t{:6s}\t{:6s}\t{:6s}\t{}".format("Word", "Lemma", "POS", "NER", "CorefMentionIndex", "CorefClusterID"))
  ner_person = set()
  # sent11=[]
  prepo = []
  verbs = []
  verbs_final=[]
  prepo_final=[]
  #prepo-result=[]
  for i, sent in enumerate(document.sentence):
    prepo = []
    verbs = []
    if verbose:
        print("[Sentence {}]".format(i+1))
        
        for t in sent.token:
          if(t.ner=="PERSON"):
            ner_person.add(t.word)        
          if(t.pos=="IN"):   # added this part
            prepo.append(t.word) # added this part
          if((t.pos =="VB")or(t.pos == "VBD")or(t.pos == "VBN") or(t.pos == "VBZ") or (t.pos =="VBP") or(t.pos == "VBG")):
            verbs.append(t.word)
        verbs_final.append(verbs)
        prepo_final.append(prepo)
      #prepo-result.append(i,)      
    print("next part")
    #follow this syntax #result.append((s,np,vp))
    #totalphrase.append((sent1,prepo)) # this works but gives same values . It is duplication, no use
    ##print(totalphrase)       
                                 
    if verbose:
      #print("{:12s}\t{:12s}\t{:6s}\t{:15s}\t{:}\t\t{}".format(t.word, t.lemma, t.pos, t.ner,t.corefMentionIndex, t.corefClusterID))
      print (" prepo, and verbs = ",i+1, prepo, verbs)
      
      # print(sentence[i])    tried but doesnot work
      #print(prepo[0])
      #print(verbs[3])
                    
  # return (SENTENCES_LENGTH,ner_person)
  return (verbs_final,prepo_final)

In [ ]:
#inputFile = "/content/Dataset1-science-sixthstandard.txt"

#outputFile = "Output-science-sixthstandard.json"  
inputFile = "/content/Data2-mctest-set3.txt"

outputFile = "/content/output-Dataset2-mctest-stories.json"

In [ ]:
def getUpdatedParagraphFile(inputFile,outputFile,verbose):
    
  document,text,sentence_np = getAnnotatedDocument(inputFile)
  #for t.text in text:   didnt work
    #print(t.text)
  #prepos = getprepos(document,text)
  print("execution of getupdateparagraphFile")
  print("printing text value")
  print(text[0])
  print(text[1])
  print("NPs ...................")
  print(sentence_np)
  
  
  # SENTENCES_LENGTH,noun_chunks = getLengthSentencesAndNouns(document,verbose)
  v,p=getLengthSentencesAndNouns(document,verbose) #should op list of list of verbs per sent
  print("---VERBS---",v)
  print("---PREPO---",p)
  print("\nInput Text : ")
  return (v,p)
   

In [ ]:
#def getprepos(document,text):
  #result1=[]
  #for i, sent in enumerate(document.sentence):

#for s in sentence:
  #pp = [t.text for t in pp]
  #result1.append(pp)
#return ((s,pp))

In [ ]:
#from ReferenceModule import References
#references = References(inputFile,outputFile).getUpdatedParagraphFile(verbose=False)
#len(references)

In [ ]:
#outputFile = "Output-science-sixthstandard.json"

In [ ]:
#with open('Output-science-sixthstandard.json','w') as final:
    #json.dump(verb_phrases,final)


In [ ]:
verb_phrases_temp,prepo_phrases_temp=getUpdatedParagraphFile(inputFile,outputFile,True)


Sentences :  ['Once upon a time I had a dog named Toodles.', 'He was black and white and had long floppy ears.', 'He also had very short legs, but really big paws.', "Every Saturday we would go to the park and play Toodles' favorite game.", 'Toodles loved playing fetch.', 'One Saturday, Toodles ran over to the pond because he saw ducks swimming there.', 'He ran all around the pond, barking at the ducks.', 'The ducks ignored him, and kept swimming.', 'Toodles was not having it!', 'He jumped into the pond and started swimming toward the ducks, chasing around his new playmates.', 'One of the ducks, braver than the others, poked Toodles with his beak - and then bit him right on one of his floppy ears!', 'Toodles barked and ran out of the pond because the duck hurt his ear.', 'Soaking wet, he ran toward where I was eating a sandwich on the grass and curled right up in my lap', 'so I could make him feel better.', 'After that, whenever he would see a duck, Toodles would run the other way.', '

2022-05-10 07:40:47 INFO: Writing properties to tmp file: corenlp_server-5e6214be03254040.props
2022-05-10 07:40:47 INFO: Starting server with command: java -Xmx5G -cp ./corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9001 -timeout 80000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-5e6214be03254040.props -annotators tokenize,ssplit,pos,lemma,ner,coref -preload -outputFormat serialized


Starting server with the Python...
Closing server...
execution of getupdateparagraphFile
printing text value
O
n
NPs ...................
[('Once upon a time I had a dog named Toodles.', ['a time', 'I', 'a dog', 'Toodles']), ('He was black and white and had long floppy ears.', ['He', 'long floppy ears']), ('He also had very short legs, but really big paws.', ['He', 'very short legs', 'really big paws']), ("Every Saturday we would go to the park and play Toodles' favorite game.", ['we', 'the park', "Toodles' favorite game"]), ('Toodles loved playing fetch.', ['Toodles', 'playing fetch']), ('One Saturday, Toodles ran over to the pond because he saw ducks swimming there.', ['Toodles', 'the pond', 'he', 'ducks']), ('He ran all around the pond, barking at the ducks.', ['He', 'the pond', 'the ducks']), ('The ducks ignored him, and kept swimming.', ['The ducks', 'him']), ('Toodles was not having it!', ['Toodles', 'it']), ('He jumped into the pond and started swimming toward the ducks, chasing 

In [ ]:
from PhraseModule import Phrases
phrases = Phrases(inputFile).generate_phrases()
len(phrases)

346

In [ ]:
phrases
with open('phrases-Output-Data2-mctest-set3.json','w') as finalphrases:
    json.dump(phrases,finalphrases)



In [ ]:
phrases

[('Once upon a time I had a dog named Toodles.',
  ['a time', 'I', 'a dog', 'Toodles'],
  ['had', 'named']),
 ('He was black and white and had long floppy ears.',
  ['He', 'long floppy ears'],
  ['was', 'had']),
 ('He also had very short legs, but really big paws.',
  ['He', 'very short legs', 'really big paws'],
  ['had very']),
 ("Every Saturday we would go to the park and play Toodles' favorite game.",
  ['we', 'the park', "Toodles' favorite game"],
  ['would go']),
 ('Toodles loved playing fetch.',
  ['Toodles', 'playing fetch'],
  ['loved playing']),
 ('One Saturday, Toodles ran over to the pond because he saw ducks swimming there.',
  ['Toodles', 'the pond', 'he', 'ducks'],
  ['ran', 'saw', 'swimming there']),
 ('He ran all around the pond, barking at the ducks.',
  ['He', 'the pond', 'the ducks'],
  ['ran', 'barking']),
 ('The ducks ignored him, and kept swimming.',
  ['The ducks', 'him'],
  ['ignored', 'kept']),
 ('Toodles was not having it!', ['Toodles', 'it'], ['was not havin

In [ ]:
# 1 makes(the two strong teeth, us)
# 2 must be taken(minerals, teeth) nsubj   teeth ojbect 
# 3 must be taken(minerals, our bones)
# 4 make(minerals,our bones)
# 5 meke(minerals,teeth)

In [ ]:
sentences = [x[0] for x in phrases]
noun_phrases = [','.join(x[1]) for x in phrases]
verb_phrases = [','.join(x[2]) for x in phrases]



In [ ]:
# verb_phrases_temp,prepo_phrases_temp
print(verb_phrases_temp)
verb_phrases2 = [','.join(x) for x in verb_phrases_temp]
print(verb_phrases2)

print(prepo_phrases_temp)
prepo_phrases = [','.join(x) for x in prepo_phrases_temp]
print(prepo_phrases)


[['are'], ['are', 'eat', 'differing'], ['Let', 'learn'], ['is'], ['is', 'fulfilled'], ['includes', 'contain'], ['are'], ['need', 'repairing'], ['get'], ['is', 'fulfilled'], ['get', 'eat'], ['is', 'measured'], ['is', 'measured'], ['improve', 'processes', 'needs'], ['obtain'], ['require', 'lead'], ['Let', 'learn'], ['needs'], ['are', 'called'], ['gives', 'provides', 'caused'], ['is', 'strengthen'], ['need', 'repairing'], ['get'], ['improve', 'processes'], ['obtain'], ['is'], ['is', 'fulfilled'], ['includes', 'contain'], ['are'], ['turns', 'comes'], ['infer', 'is'], ['contain'], ['contain'], ['obtain'], ['form'], ['get'], ['uses'], [], ['are'], ['are', 'eat', 'differing'], ['Let', 'learn'], ['giving', 'Carbohydrates', 'living', 'taking', 'using', 'is', 'called', 'are', 'called'], ['is'], ['is', 'fulfilled'], ['includes', 'contain'], ['are'], ['taste', 'contain'], ['get', 'contains', 'called'], ['contain'], ['give'], ['make'], ['produce'], ['takes'], ['is', 'brought'], ['is', 'called'], ['

In [ ]:
#print(posTag(sentences[0]))  change here to get preposition tags

In [ ]:
import pandas as pd
df = pd.DataFrame({'Sentence': sentences , 'NounChunks': noun_phrases, 'Verb/VP':verb_phrases2, 'Proposition' : prepo_phrases})

ValueError: ignored

In [ ]:
df

In [ ]:
df.to_csv('Dataframep1.csv',index=False)

In [ ]:
sentences

['There are six main nutrients in our food : Carbohydrates, fats, proteins, fibre, vitamins and minerals.',
 'These nutrients are present in the different food items we eat in differing proportions.',
 'Let us learn more about some of them.',
 'Our main need is of energy.',
 'It is fulfilled by carbohydrates.',
 'Hence,  our  diet  includes a large proportion of cereals in the form of rice, chapatti or roti, bhakri and bread which contain carbohydrates.',
 'Cereals are our staple food.',
 'Proteins We need proteins for the purposes of growth, repairing the wear and tear of the body and for other life processes.',
 'We get proteins from the sprouts, milk and milk products, meat and eggs in our food.',
 'Fats : Some part of our energy requirement is also fulfilled by fatty foods such as oil, ghee and butter.',
 'We get energy in the form of heat from the food we eat.',
 'Heat is measured in kilocalories.',
 'Hence, the energy in food items is also measured in kilocalories (Calories).',
 